<img src="https://github.com/brazil-data-cube/code-gallery/blob/master/img/logo-bdc.png?raw=true" align="right" width="110" />


# <span style="color:#336699"> Sensoriamento Remoto aplicado ao estudo de ecossistemas de água doce</span>
<hr style="border:2px solid #0077b9;">

<br>

<div style="text-align: center;font-size: 90%;">
    Daniel Andrade Maciel<sup><a href="http://www.dpi.inpe.br/labisa/authors/daniel_maciel/"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>
    <br/><br/>
    Earth Observation and Geoinformatics Division, National Institute for Space Research (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    Contact: <a href="mailto:daniel.maciel@inpe.br">daniel.maciel@inpe.br</a>
    <br/><br/>
    Last Update: May, 2025
</div>

<br>

<div style="text-align: justify;  margin-left: 25%; margin-right: 25%;">
<b>Abstract.</b> O uso de dados de sensoriamento remoto para prever parâmetros de qualidade da água (QA) que são opticamente ativos (ou seja, que interagem com a luz) tem sido aplicado a águas oceânicas e costeiras há cerca de 50 anos. Graças à nova geração de sensores com resolução espectral, radiométrica e espacial adequadas (como Landsat, Sentinel-2, etc.), nos últimos 15 anos a comunidade científica começou a aplicar o sensoriamento remoto (SR) em estudos de águas interiores. O sensoriamento remoto permite prever alguns parâmetros de qualidade da água: sedimentos em suspensão, clorofila-a, ficocianina, matéria orgânica dissolvida, carbono, profundidade do disco de Secchi, turbidez... É uma fonte importante de dados que pode auxiliar biólogos, limnólogos e toda a comunidade de ciências aquáticas na compreensão dos padrões da água. Neste workshop, vamos aprender como usar dados de Sensoriamento Remoto aplicados às ciências aquáticas. Utilizaremos dados in situ disponíveis no conjunto de dados GLORIA (Lehmann et al. 2023) para gerar um modelo empírico de concentração de clorofila-a com base no Índice de Diferença Normalizada da Clorofila-a (NDCI). Assim, com um algoritmo calibrado, aplicaremos o modeloo desenvolvido aos dados de Reflectância de Superfície do Sentinel-2/MSI, corrigidos atmosfericamente usando o Sen2Cor e disponíveis na plataforma STAC da Microsoft Planetary Computer. Iremos também aprender como realizar o downl

O fluxo de processamento está dividido em três tópicos:

1) Instalação dos pacotes, download dos dados, simulação das bandas e remoção de outliers (etapa de pré-processamento);

2) Desenvolvimento do modelo (treinamento, validação e desenvolvimento do modelo final);

3) Aplicação do modelo: aplicação dos algoritmos aos dados de satélite usando o STAC do Brazil Data Cube

4) Aplicação da correção atmosférica no nível L1C e comparação com os resultados do Sen2Cor.

## O que esperamos obter como resultado?

1) Um algoritmo de clorofila-a utilizando um model empírico (NDCI) (Veja mais informações em Lobo et al. (2020) (https://www.mdpi.com/2072-4292/13/15/2874);

2) Aplicação deste algoritmo (NDCI) em uma imagem Sentinel-2/MSI obtida através do STAC do Brazil Data Cube (BDC) no nível 2A;

3) Introdução a correção atmosférica de imagens para ambientes aquáticos com o ACOLITE (Vanhellemont and Ruddick, 2018) (https://www.sciencedirect.com/science/article/pii/S0034425718303481);

4) Download e aplicação da correção atmosférica ACOLITE em dados Sentinel-2 L1C obtidos através do STAC do BDC

5) Comparação dos resultados entre os dois métodos


</em>.
</div>    

   



## <span style="color: #336699">Parte 01 - Pré-processamento e download de dados</span>
<hr style="border:0.5px solid #0077b9;">

<br>


In [ ]:
## Set directory

## Instalar pacotes necessários - simulação de bandas

devtools::install_github("dmaciel123/BandSimulation")

require(bandSimulation)

options(warn = -1)
options(repr.plot.width = 15, repr.plot.height = 15) 



In [ ]:
require(data.table)
require(dplyr)
require(PerformanceAnalytics)


In [ ]:
## Configure the Project Directories 

dir.create("Data")
dir.create("Outputs")
dir.create("Scripts")

# GLORIA Dataset

O conjunto de dados GLORIA é uma compilação de dados de reflectância de sensoriamento remoto (Rrs) e qualidade da água para corpos d’água em escala global, com dados dedicados a ecossistemas de água doce. É gratuito, acessível a todos e cobre grande parte do planeta, com mais de 7.000 amostras (Figura 01).

Vale lembrar que a Reflectância de Sensoriamento Remoto (Rrs) é a razão entre a radiância emergente da água e a irradiância descendente, compensada pela radiância do céu e corrigida pelos efeitos de brilho especular (glint) (Equação 01).

Para mais informações, consulte a publicação [(Lehmann et al. 2023)](https://www.nature.com/articles/s41597-023-01973-y) e o dataset disponível no [PANGAEA](http://https://doi.pangaea.de/10.1594/PANGAEA.948492) e [Nature Earth and Environmment blog post](https://communities.springernature.com/posts/gloria-challenges-in-developing-a-globally-representative-hyperspectral-in-situ-dataset-for-the-remote-sensing-of-water-resources)



![Figure 01](https://earthenvironmentcommunity.nature.com/cdn-cgi/image/metadata=copyright,fit=scale-down,format=auto,sharpen=1,quality=95/https://images.zapnito.com/uploads/hiCMOprnTtSCTJNv78gu_locations.jpg)


In [ ]:

###### Download GLORIA data ##########

URL = 'https://download.pangaea.de/dataset/948492/files/GLORIA-2022.zip'

# Before download, let`s set timeout to 200s (sometimes PANGAEA download is slow)

options(timeout=300)

# If the directory with files doesn't exist, let's download GLORIA data.

if(dir.exists('Data/GLORIA_2022/') == FALSE) {
  
  # Download
  download.file(URL, 'Data/GLORIA.zip')
  
  # Extract
  unzip(zipfile = 'Data/GLORIA.zip', exdir = 'Data')
  
}

In [ ]:

##### Analyzing GLORIA data #######

meta_and_lab = fread("Data/GLORIA_2022/GLORIA_meta_and_lab.csv")
rrs = fread("Data/GLORIA_2022/GLORIA_Rrs.csv")

##### Plot for different concentrations #######

# High Chl-a

par(mfrow=c(2,2))

matplot(t(select(rrs, paste("Rrs_", 400:900, sep = ''))[rrs$GLORIA_ID == 'GID_7403',]), ylim = c(0,0.06),
        x= c(400:900), pch = 20, xlab = '', ylab = '', type = 'l', main = "Alta Clorofila", cex.main = 2 )

# High TSS

matplot(t(select(rrs, paste("Rrs_", 400:900, sep = ''))[rrs$GLORIA_ID == 'GID_1805',]), ylim = c(0,0.06),
        x= c(400:900), pch = 20, xlab = '', ylab = '', type = 'l', main = "Alto Sedimento" , cex.main = 2)


# High aCDOM

matplot(t(select(rrs, paste("Rrs_", 400:900, sep = ''))[rrs$GLORIA_ID == 'GID_2468',]), ylim = c(0,0.0005),
        x= c(400:900), pch = 20, xlab = '', ylab = '', type = 'l', main = "Alto aCDOM", cex.main = 2)


# Simulação de Bandas

Quando simulamos uma banda de satélite, estamos compensando as diferenças na sensibilidade dos detectores a cada comprimento de onda. A figura abaixo mostra as diferenças na função de resposta espectral para os sensores Sentinel-2A/MSI, Landsat-8/OLI e Landsat-7/ETM+. É possível notar que valores de resposta espectral relativa próximos de "1" indicam que o detector consegue medir (ou detectar) toda a radiância naquele comprimento de onda.

Uma banda de um sensor é composta por um intervalo desses comprimentos de onda e, portanto, a banda simulada é a integração da R[rs] considerando a curva de Resposta Espectral Relativa.

Deixar só do sentinel

![Figure 02](https://upload.wikimedia.org/wikipedia/commons/7/7d/Spectral_responses_of_Landsat_7_ETM%2B%2C_Landsat_8_OLI_and_Sentinel_2_MSI_in_the_visible_and_near_infrared.png)


In [ ]:
require(bandSimulation)

# Selecionar as bandas entre 400 e 900 nm e transpor (Wavelength tem que ser nas linhas)
spectra_formated = select(rrs, paste("Rrs_", 400:900, sep = '')) %>% t()

# Simulação para MSI
MSI_sim = msi_simulation(spectra = spectra_formated, 
                         point_name = rrs$GLORIA_ID)


#It simulates for Sentinel-2A and Sentinel-2B and gives the results in a list.
# Let's select only Sentinel-2A.

# Pegando os dados do Sentinel-2/A
MSI = MSI_sim$s2a[,-1] %>% t() %>% data.frame()


# Add names to a collumn
MSI$GLORIA_ID = row.names(MSI)

# Change band names

names(MSI) = c('x440', "x490", 'x560', 'x660', "x705", 'x740', 'x780', 'x850', 'x865', "GLORIA_ID")


selection = filter(rrs, GLORIA_ID == 'GID_207')
selection.s = filter(MSI, GLORIA_ID == 'GID_207')
meta.s = filter(meta_and_lab, GLORIA_ID == 'GID_207')


In [ ]:

##### Plot example #####

par(mar = c(8,8,8,8))
matplot(t(select(selection, paste("Rrs_", 400:900, sep = ''))[,]), ylim = c(0,0.01),
        x= c(400:900), pch = 20, xlab = '', ylab = '', cex.axis = 2, cex.lab = 2, cex = 2)

par(new=T)

matplot(t(selection.s[,-10]), x= c(440,490,560,660,705,740,780,842,860), pch = 20,
        ylim = c(0,0.01), xlim = c(400,900), col = 'red', , xlab = 'Wavelength (nm)', 
        ylab = 'Rrs (sr-1)', cex.axis = 2, cex.lab = 2, cex = 2)

legend('topleft', legend = c(paste("Chl-a = ", meta.s$Chla),
                             paste("Secchi = ", meta.s$Secchi_depth)), cex = 2)


In [ ]:
## Como eu tenho duas planilhas separadas, vamos fazer uma junção das duas ####

## Merge with water quality, lat long (By GLORIA_ID)

merged = merge(select(meta_and_lab, c('GLORIA_ID', 'Chla', "Latitude", "Longitude")),
               MSI, by = "GLORIA_ID")

###### Index calculation and NA remove #######

# Remover os NAs

merged = merged[is.na(merged$Chla) == FALSE, ]

# Remover valores muito altos > 1000 e menor/igual a zero

merged = merged[(merged$Chla < 1000 & merged$Chla > 0), ]

# Calculo do NDCI

## Adicionra +1 para o mmodelo de potencial - veja em Lobo et al. (2021)

merged$NDCI = ((merged$x705-merged$x660)/(merged$x705+merged$x660))+1

merged = filter(merged, NDCI > 0 & NDCI < 2)

# Correlations

chart.Correlation((select(merged, c("Chla", 
                                   "NDCI"))))

In [ ]:
write.table(merged, file = 'Outputs/sentinel2_simulated_filtered.csv', row.names = F)

In [ ]:
getwd()

